In [3]:
pip install python-chess

  Using cached python_chess-1.999-py3-none-any.whl.metadata (776 bytes)
  Using cached chess-1.10.0-py3-none-any.whl.metadata (19 kB)
Using cached python_chess-1.999-py3-none-any.whl (1.4 kB)
Using cached chess-1.10.0-py3-none-any.whl (154 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import chess.pgn
import chess.engine

def eval_to_score(eval):
    if eval.is_mate():
        mate_in = eval.mate()
        # Assign a large positive or negative value for mate scores
        if mate_in > 0:
            return 100000  # Mate in N moves (winning)
        else:
            return -100000  # Getting mated in N moves (losing)
    else:
        return eval.score()

def analyze_single_game(pgn_file, engine_path):
    # Load the PGN file
    with open(pgn_file) as pgn:
        game = chess.pgn.read_game(pgn)

    if game is None:
        print("No game found in the PGN file.")
        return

    # Load the chess engine
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)

    board = game.board()  # Set up the initial board position

    # Extract the Elo ratings and the result from the headers
    white_elo = game.headers.get("WhiteElo", "N/A")
    black_elo = game.headers.get("BlackElo", "N/A")
    result = game.headers.get("Result", "*")

    print(f"White Elo: {white_elo}, Black Elo: {black_elo}, Result: {result}")

    move_number = 1  # Track move number
    white_cpls = []
    black_cpls = []

    node = game  # Start from the root node

    while not node.is_end():
        try:
            # Get the next move
            next_node = node.variation(0)
            move = next_node.move

            # Get SAN notation before pushing the move
            move_san = board.san(move)

            # Determine whose turn it is
            player_color = board.turn  # True for White, False for Black
            player = 'White' if player_color else 'Black'
            color_factor = 1 if player_color else -1

            # Get the engine's recommended move and its evaluation
            info = engine.analyse(board, chess.engine.Limit(depth=12))
            best_move = info["pv"][0]
            eval_best = info["score"].white()

            # Apply the best move on a copy of the board
            board_best = board.copy()
            board_best.push(best_move)
            info_best = engine.analyse(board_best, chess.engine.Limit(depth=0))
            eval_after_best = info_best["score"].white()

            # Apply the player's move to the board
            board.push(move)

            # Get the evaluation after the player's move
            info_after = engine.analyse(board, chess.engine.Limit(depth=0))
            eval_after_move = info_after["score"].white()

            # Convert evaluations to centipawn scores
            eval_after_best_cp = eval_to_score(eval_after_best)
            eval_after_move_cp = eval_to_score(eval_after_move)

            # Compute CPL = (eval_after_best - eval_after_move) * color_factor
            delta_e = (eval_after_best_cp - eval_after_move_cp) * color_factor

            # If delta_e negative, set to zero
            if delta_e < 0:
                delta_e = 0

            # Append delta_e to the appropriate list
            if player_color:
                white_cpls.append(delta_e)
            else:
                black_cpls.append(delta_e)

            print(f"Move {move_number}: {player} played {move_san}, CPL: {delta_e / 100:.2f} pawns")

            move_number += 1
            node = next_node

        except Exception as e:
            print(f"Error processing move {move_number}: {str(e)}")
            break

    engine.quit()

    # Compute ACPL for each player
    if white_cpls:
        white_acpl = sum(white_cpls) / len(white_cpls) / 100  # Convert to pawns
    else:
        white_acpl = None

    if black_cpls:
        black_acpl = sum(black_cpls) / len(black_cpls) / 100  # Convert to pawns
    else:
        black_acpl = None

    print(f"\nWhite ACPL: {white_acpl:.2f} pawns")
    print(f"Black ACPL: {black_acpl:.2f} pawns")

# Path to the Stockfish engine
stockfish_path = r"C:\Users\foivo\Downloads\stockfish-windows-2022-x86-64-avx2\stockfish.exe"

pgn_file = "example.pgn"  # The PGN file with your game

# Analyze the single game
analyze_single_game(pgn_file, stockfish_path)


FileNotFoundError: [WinError 2] The system cannot find the file specified